In [1]:
from dolfin import *
import numpy as np
import pygmsh as pg
import meshio
import matplotlib.pyplot as plt

In [2]:
# Create mesh and define function space
geom = pg.opencascade.Geometry(
  characteristic_length_min=0.4,
  characteristic_length_max=0.4,
  )
rectangle = geom.add_rectangle([-1.0, -1.0, 0], 2.0, 2.0)
# disk = geom.add_disk([0.0, 0.0, 0], 0.25)
# solid = geom.boolean_difference([rectangle], [disk])

mesh = pg.generate_mesh(geom)
meshio.write('Geometry/Circular-inclusion.xml', mesh)

Info    : Running 'gmsh -3 /tmp/tmpa9dj26a0.geo -format msh -bin -o /tmp/tmpwh97x6qu.msh' [Gmsh 3.0.6, 1 node, max. 1 thread]
Info    : Started on Mon Aug  3 18:15:43 2020
Info    : Reading '/tmp/tmpa9dj26a0.geo'...
Info    : Done reading '/tmp/tmpa9dj26a0.geo'
Info    : Finalized high order topology of periodic connections
Info    : Meshing 1D...
Info    : Meshing curve 1 (Line)
Info    : Meshing curve 2 (Line)
Info    : Meshing curve 3 (Line)
Info    : Meshing curve 4 (Line)
Info    : Done meshing 1D (0.000236 s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Delaunay)
Info    : Done meshing 2D (0.01248 s)
Info    : Meshing 3D...
Info    : Done meshing 3D (5e-06 s)
Info    : 45 vertices 92 elements
Info    : Writing '/tmp/tmpwh97x6qu.msh'...
Info    : Done writing '/tmp/tmpwh97x6qu.msh'
Info    : Stopped on Mon Aug  3 18:15:43 2020


In [7]:
mesh = Mesh("Geometry/Circular-inclusion.xml")

V = FunctionSpace(mesh, 'Lagrange', 2)

mesh

In [8]:
# Define boundary condition
u_D = Expression('1 + x[0]*x[0] + 2*x[1]*x[1]', degree=2)

def boundary(x, on_boundary):
    return on_boundary
bc = DirichletBC(V, u_D, boundary)

In [11]:
# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = Constant(-6.0)
a = dot(grad(u), grad(v))*dx
L = f * v * dx

In [12]:
# Compute solution
u = Function(V)
solve(a == L, u, bc)

In [13]:
# Compute error in L2 norm
error_L2 = errornorm(u_D, u, 'L2')

In [14]:
# Compute maximum error at verticies
vertex_values_u_D = u_D.compute_vertex_values(mesh)
vertex_values_u = u.compute_vertex_values(mesh)

In [15]:
import numpy as np
error_max = np.max(np.abs(vertex_values_u - vertex_values_u_D))
print('u_exact = ', vertex_values_u_D)
print('u_fem = ', vertex_values_u)

u_exact =  [4.         4.         4.         4.         3.36       3.04
 3.04       3.36       2.72       2.08       2.08       2.72
 3.36       3.04       3.04       3.36       2.72       2.08
 2.08       2.72       1.         1.541875   1.541875   1.541875
 1.541875   1.35146967 1.7016598  1.7016598  1.35146967 2.35102759
 2.35102759 2.35102759 2.35102759 1.10814921 1.15726289 1.1319479
 1.10022181 1.82549117 2.22891676 1.82549117 2.22891676 1.83802176
 2.23518205 1.83802176 2.23518205]
u_fem =  [4.         4.         4.         4.         3.36       3.04
 3.04       3.36       2.72       2.08       2.08       2.72
 3.36       3.04       3.04       3.36       2.72       2.08
 2.08       2.72       1.         1.541875   1.541875   1.541875
 1.541875   1.35146967 1.7016598  1.7016598  1.35146967 2.35102759
 2.35102759 2.35102759 2.35102759 1.10814921 1.15726289 1.1319479
 1.10022181 1.82549117 2.22891676 1.82549117 2.22891676 1.83802176
 2.23518205 1.83802176 2.23518205]


In [16]:
# Print errors
print('error L2 = ', error_L2)
print('error_max = ', error_max)

error L2 =  3.752295771864171e-14
error_max =  3.352873534367973e-14


Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', Cell('triangle', 3), 1), dim=3), 13), FiniteElement('Lagrange', Cell('triangle', 3), 2)), 36)